### 轉換棋局至array

In [ ]:
import numpy as np
import chess

def board_to_array(board):
    # Initialize an empty array to represent the board
    board_array = np.zeros((8, 8, 12), dtype=np.float32)
    
    # Mapping of piece types to index in the third dimension of the board_array
    piece_idx = {'p': 0, 'P': 6, 'n': 1, 'N': 7, 'b': 2, 'B': 8, 'r': 3, 'R': 9, 'q': 4, 'Q': 10, 'k': 5, 'K': 11}
    
    for i in range(8):
        for j in range(8):
            piece = board.piece_at(chess.square(i, 7-j))
            if piece:
                board_array[j, i, piece_idx[piece.symbol()]] = 1.0
    
    return board_array

### 處理真實對局

In [ ]:
import numpy as np

# Function to save data
def save_data(data, labels, file_name):
    np.savez(file_name, data=data, labels=labels)

# Function to load data
def load_data(file_name):
    loaded_data = np.load(file_name)
    return loaded_data['data'], loaded_data['labels']

In [ ]:
import chess.pgn
import os

def convert_data_from_realword(path = "data/chess_raw"):
    data = []
    labels = []
    filenames = [f for f in os.listdir(path) if f.endswith(".pgn")]
    for filename in filenames:
        pgn = open(f"{path}/{filename}")
        while True:
            game = chess.pgn.read_game(pgn)
            if game is None:
                break  # End of file
            board = game.board()
            for move in game.mainline_moves():
                board.push(move)
                board_array = board_to_array(board)
                label = 1.0 if board.turn == chess.WHITE else 0.0  # 1 for white's turn, 0 for black
                data.append(board_array)
                labels.append(label)
    
    # Adjust dimensions
    data = np.transpose(data, (0, 3, 1, 2))
    save_data(np.array(data), np.array(labels), "./data/real_cases.npz")
    return np.array(data), np.array(labels)

### 隨機產生棋局

In [ ]:
def generate_data(num_games):
    data = []
    labels = []
    for _ in range(num_games):
        board = chess.Board()
        while not board.is_game_over():
            move = np.random.choice([m for m in board.legal_moves])
            board.push(move)
            board_array = board_to_array(board)
            label = 1.0 if board.turn == chess.WHITE else 0.0  # 1 for white's turn, 0 for black
            data.append(board_array)
            labels.append(label)
    
    # Adjust dimensions
    data = np.transpose(data, (0, 3, 1, 2))
    return np.array(data), np.array(labels)

### 實際使用

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from lightning import LightningModule, Trainer

torch.set_float32_matmul_precision("high")

class ChessCNN(LightningModule):
    def __init__(self):
        super(ChessCNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(12, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 1)
        
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.BCELoss()(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.BCELoss()(y_hat, y)
        self.log('val_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.BCELoss()(y_hat, y)
        self.log('test_loss', loss)
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)
    
# Generate some training data
train_data, train_labels = generate_data()

# Convert to PyTorch tensors
train_data = torch.tensor(train_data).float()
train_labels = torch.tensor(train_labels).float().view(-1, 1)

# Create a DataLoader
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(train_data, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize and train the model
model = ChessCNN()
trainer = Trainer(max_epochs=10)
trainer.fit(model, train_loader)
# Assuming `trainer` is your Trainer object
trainer.save_checkpoint("chess_model.ckpt")

In [ ]:
import io
import torch

def choose_move(model, board):
    best_move = None
    best_value = -1.0  # Initialize with a low value

    # Iterate through all legal moves
    for move in board.legal_moves:
        board.push(move)
        board_array = board_to_array(board)
        board_array = np.transpose(board_array, (2, 0, 1))  # Adjust dimensions to match the model input
        board_array = torch.tensor(board_array).float().unsqueeze(0)  # Add batch dimension
        value = model(board_array).item()
        board.pop()  # Revert the move to go back to the original state

        if value > best_value:
            best_value = value
            best_move = move

    return best_move

board = chess.Board()
model = ChessCNN.load_from_checkpoint("chess_model.ckpt")
# move = choose_move(model, board)
# print("Best move according to the model:", move)
while not board.is_game_over():
    move = choose_move(model, board)
    board.push(move)
print(board)

# Generate an SVG of the board
board_svg = chess.svg.board(board=board)

# Convert the SVG to a PIL image
image = Image.open(io.BytesIO(board_svg.encode('utf-8')))
image.show()
